<a href="https://colab.research.google.com/github/UmarIgan/Machine-Learning/blob/master/Fine_Tune_LLaMA_2_with_QLoRA_for_QA_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-Tune LLaMA 2 with QLoRA
By Matt Shumer (https://twitter.com/mattshumer_)

Fine-tune LLaMA 2 models w/ very low resource usage.

Use the best GPU available (go to Runtime -> change runtime type

To fine-tune a model, just load in a JSONL file `train.jsonl` with `prompt` and `response` keys, and do the same for `test.jsonl`. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

Heavy credit goes to https://mlabonne.github.io/blog/posts/Fine_Tune_Your_Own_Llama_2_Model_in_a_Colab_Notebook.html... this is just a slightly modified version that adds a few extra features and supports OpenAI-like jsonl datasets.

# Install&Import libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


# Define Hyperparameters

In [2]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "/content/finqa"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 5
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

#Load Datasets

In [3]:
train_dataset = load_dataset("dreamerdeo/finqa", split = 'train').remove_columns(["post_text", 'gold_evidence', 'table'])
test_dataset = load_dataset("dreamerdeo/finqa", split = 'test').remove_columns(["post_text", 'gold_evidence', 'table'])

Repo card metadata block was not found. Setting CardData to empty.


/root/.cache/huggingface/datasets/downloads/extracted/ca045c27488b49db24426c67e0b35e06b0188746c40ffaf8d01a7b51e52924dc


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
def add_prefix(example):
    example["text"] = ('Follow the given context: ' + ' '.join([str(item) for item in example['pre_text']]) + 'This is the question about this context: ' + example["question"] + 'and this is the answer: ' +  example["answer"])
    return example

In [5]:
train_dataset.features

{'id': Value(dtype='string', id=None),
 'pre_text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None)}

In [6]:
test_dataset.features

{'id': Value(dtype='string', id=None),
 'pre_text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None)}

In [10]:
from huggingface_hub import login
login()

# Training

In [8]:
# Preprocess datasets
train_dataset = train_dataset.map(add_prefix)
test_dataset = test_dataset.map(add_prefix)

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
    )

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

Map:   0%|          | 0/1147 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=20  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
#trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

Map:   0%|          | 0/1147 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20,1.723200,1.618990
40,1.496900,1.406386
60,1.373700,1.347506
80,1.328800,1.316707
100,1.209200,1.333116
120,1.310900,1.299648
140,1.340600,1.288884
160,1.339700,1.291473
180,1.145300,1.274379
200,1.184200,1.289692


Step,Training Loss,Validation Loss
20,1.723200,1.618990
40,1.496900,1.406386
60,1.373700,1.347506
80,1.328800,1.316707
100,1.209200,1.333116
120,1.310900,1.299648
140,1.340600,1.288884
160,1.339700,1.291473
180,1.145300,1.274379
200,1.184200,1.289692


TrainOutput(global_step=1563, training_loss=1.1542937238477242, metrics={'train_runtime': 30652.8442, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.051, 'total_flos': 6.139726357868544e+16, 'train_loss': 1.1542937238477242, 'epoch': 1.0})

In [7]:
#trainer.model.save_pretrained(new_model) - uncomment your option
#trainer.push_to_hub()

# Testing new model.

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("umarigan/results")
model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf")#had some problem with meta model
model = PeftModel.from_pretrained(model, "umarigan/results")

In [1]:
actual_valid_dataset = load_dataset("dreamerdeo/finqa", split = 'validation')

NameError: ignored

In [2]:
actual_valid_dataset["pre_text"][:3]

NameError: ignored

In [ ]:
actual_valid_dataset["question"][:3]

In [ ]:
actual_valid_dataset["answer"][:3]

In [ ]:
question_answerer = pipeline("question-answering")
context = """
performance graph comparison of five-year cumulative total return the following graph and table compare the cumulative
total return on citi 2019s common stock , which is listed on the nyse under the ticker symbol 201cc 201d and held by
65691 common stockholders of record as of january 31 , 2018 , with the cumulative total return of the s&p 500 index
and the s&p financial index over the five-year period through december 31 , 2017 .", "the graph and table assume
that $ 100 was invested on december 31 , 2012 in citi 2019s common stock , the s&p 500 index and the s&p financial
index , and that all dividends were reinvested .", "comparison of five-year cumulative total return for the years
ended date citi s&p 500 financials .
"""
question = "what was the percentage cumulative total return for the five year period ended 31-dec-2017 of citi common stock?"
question_answerer(question=question, context=context, model=model, tokenizer=tokenizer, max_length=200)#answer is 93.5%